In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputRegressor

In [4]:
df = pd.read_excel("C:\\Users\\masali\\OneDrive - Wiley\\Desktop\\FYGoals\\Data for FY2026 06242025.xlsx", header=1)

In [7]:
df.head()

,FY,journal code,Publishing Development Group,IP Owner,Owned By,WOL Level 2 Subject,Editorial Office Model,Editor In Chief Model,ownership,revenue model,...,marketing categorization,Submissions,Decisions,Final Decisions,Median Submission to First Decision TAT,Median Submission to Acceptance TAT,Acceptance Rate Including Non Peer Reviewed Papers,Acceptance Rate Including Peer Reviewed Papers,Triage Rate,EDR Rate
0,2025,ABAC,APAC-JPN/AUS/NZ,JWAS Australia Ltd. AU,"The Accounting Foundation, The University of S...",Accounting,External,External,Society Owned,Hybrid Model,...,Established Title,364.0,359.0,349.0,125.466331,328.996916,0.117479,0.310345,0.653295,NaN
1,2025,ACEM,HS2,"John Wiley & Sons, Inc. US",Society for Academic Emergency Medicine,Emergency Medicine,External,External,Society Owned,Hybrid Model,...,Established Title,1337.0,1321.0,1311.0,44.506707,85.184236,0.169336,0.571429,0.755149,0.050343
2,2025,ACFI,APAC-JPN/AUS/NZ,JWAS Australia Ltd. AU,Accounting and Finance Association of Australi...,Accounting,Vendor Managed,External,Society Owned,Hybrid Model,...,Established Title,1089.0,1078.0,984.0,68.922101,283.181944,0.168699,0.372998,0.552846,NaN
3,2025,ACR2,HS3,"John Wiley & Sons, Inc. US",American College of Rheumatology,Rheumatology,J&J Editorial,External,Society Owned,OA,...,Established Title,425.0,428.0,414.0,63.300208,136.450949,0.410628,0.754717,0.495169,0.229469
4,2025,AAS,HS1,John Wiley & Sons Ltd. UK,Scandinavian Society of Anaesthesiology and In...,Anaesthesia & Pain Management,Vendor Managed,External,Society Owned,Hybrid Model,...,Established Title,950.0,959.0,949.0,36.021626,117.538796,0.202318,0.676647,0.740780,0.618546


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6264 entries, 0 to 6263
Data columns (total 27 columns):
 #   Column                                              Non-Null Count  Dtype  
---  ------                                              --------------  -----  
 0   FY                                                  6264 non-null   int64  
 1   journal code                                        6264 non-null   object 
 2   Publishing Development Group                        6264 non-null   object 
 3   IP Owner                                            6264 non-null   object 
 4   Owned By                                            6264 non-null   object 
 5   WOL Level 2 Subject                                 6244 non-null   object 
 6   Editorial Office Model                              6264 non-null   object 
 7   Editor In Chief Model                               6096 non-null   object 
 8   ownership                                           6264 non-null   object 
 9

In [9]:
df.columns = df.columns.str.strip()
df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

In [10]:
excluded_cols = ['IP Owner', 'Owned By', 'launch year', 'Age of Journal']
df_cleaned = df.drop(columns=excluded_cols).copy()

In [11]:
num_cols = df_cleaned.select_dtypes(include=['float64', 'int64']).columns
num_cols = [col for col in num_cols if col != 'FY']

In [ ]:
for col in num_cols:
    df_cleaned[col] = df_cleaned.groupby('journal code')[col].transform(lambda grp: grp.fillna(grp.medan()))

In [12]:
df_cleaned = df_cleaned.sort_values(['journal code', 'FY'])
for lag in [1, 2, 3]:
    df_cleaned[f'Submissions_Lag{lag}'] = df_cleaned.groupby('journal code')['Submissions'].shift(lag)
df_cleaned['Submissions_Trend'] = df_cleaned['Submissions_Lag1'] - df_cleaned['Submissions_Lag3']


Preparing Data for Submission Number Prediction

In [13]:
feature_cols = ['Submissions_Lag1', 'Submissions_Lag2', 'Submissions_Lag3', 'Submissions_Trend']
train_df = df_cleaned[df_cleaned[feature_cols].notnull().all(axis=1)].copy()
X_train = train_df[train_df['FY'] < 2026][feature_cols]
y_train = train_df[train_df['FY'] < 2026]['Submissions']

Training Model and Predicting Submission for FY2026

In [14]:
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

predict_df = df_cleaned[df_cleaned['FY'] == 2025].copy()
predict_df['Submissions_Trend'] = predict_df['Submissions_Lag1'] - predict_df['Submissions_Lag3']
X_pred = predict_df[feature_cols]
predict_df['Predicted_Submissions_2026'] = rf_model.predict(X_pred).round().astype(int)

Defining Target and Categorical Column

In [16]:
target_cols = [
    'Decisions', 'Final Decisions',
    'Median Submission to First Decision TAT',
    'Median Submission to Acceptance TAT',
    'Acceptance Rate Including Non Peer Reviewed Papers',
    'Acceptance Rate Including Peer Reviewed Papers',
    'Triage Rate', 'EDR Rate'
]

categorical_cols = [
    'Publishing Development Group', 'WOL Level 2 Subject',
    'Editorial Office Model', 'Editor In Chief Model', 'ownership', 'revenue model',
    'sound science title', 'peer review model', 'free format', 'Wiley Research Exchange',
    'OPT Portfolio', 'marketing categorization'
]

Prepare data for Other Target Columns

In [17]:
model2_train_df = df_cleaned[df_cleaned['FY'] == 2025].copy()
model2_train_df = model2_train_df.dropna(subset=target_cols)

X_other = model2_train_df[categorical_cols + ['Submissions']].dropna()
y_other = model2_train_df.loc[X_other.index, target_cols]

Creating Prediction Inputs for Other Columns:

In [18]:
X_pred_other = predict_df[categorical_cols].copy()
X_pred_other['Submissions'] = predict_df['Predicted_Submissions_2026']
X_pred_other = X_pred_other.dropna()
predict_df = predict_df.loc[X_pred_other.index]

Fitting the Model for predicting other columns

In [19]:
preprocessor = ColumnTransformer(
    transformers=[('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)],
    remainder='passthrough'
)

model2 = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', MultiOutputRegressor(RandomForestRegressor(n_estimators=100, random_state=42)))
])

model2.fit(X_other, y_other)
predicted_other_metrics = model2.predict(X_pred_other)

Exporting the Final Output

In [22]:
predicted_metrics_df = pd.DataFrame(predicted_other_metrics, columns=[col + '_2026' for col in target_cols])
final_predictions = pd.concat([
    predict_df[['journal code', 'Submissions', 'Decisions', 'Predicted_Submissions_2026']].reset_index(drop=True),
    predicted_metrics_df
], axis=1)

final_predictions.to_excel("C:\\Users\\masali\\OneDrive - Wiley\\Desktop\\FYGoals\\test.xlsx", index=False)